In [1]:
import time
import os
import cPickle
import sys
import binascii
import multiprocessing as mp
from itertools import chain
import ast
import json
import subprocess
import sys
#from scapy.all import *
#sys.path.append('hed-dlg/')
import numpy as np

sys.setrecursionlimit(100000)

In [2]:
def parse_header(line):
    ret_dict = {}
    h = line.split()
    #ret_dict['direction'] = " ".join(h[3:6])
    if h[2] == 'IP6':
        """
        Conditional formatting based on ethernet type.
        IPv4 format: 0.0.0.0.port
        IPv6 format (one of many): 0:0:0:0:0:0.port
        """
        ret_dict['src_port'] = h[3].split('.')[-1]
        ret_dict['src_ip'] = h[3].split('.')[0]
        ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
        ret_dict['dest_ip'] = h[5].split('.')[0]
    else:
        if len(h[3].split('.')) > 4:
            ret_dict['src_port'] = h[3].split('.')[-1]
            ret_dict['src_ip'] = '.'.join(h[3].split('.')[:-1])
        else:
            ret_dict['src_ip'] = h[3]
            ret_dict['src_port'] = ''
        if len(h[5].split('.')) > 4:
            ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
            ret_dict['dest_ip'] = '.'.join(h[5].split('.')[:-1])
        else:
            ret_dict['dest_ip'] = h[5].split(':')[0]
            ret_dict['dest_port'] = ''
    return ret_dict

def parse_data(line):
    ret_str = ''
    h, d = line.split(':', 1)
    ret_str = d.strip().replace(' ', '')
    return ret_str

def process_packet(output):
    # TODO!! throws away the first packet!
    ret_header = {}
    ret_dict = {}
    ret_data = ''
    hasHeader = False
    for line in output:
        line = line.strip()
        if line:
            if not line.startswith('0x'):
                # header line
                if ret_dict and ret_data:
                    # about to start new header, finished with hex
                    ret_dict['data'] = ret_data
                    yield ret_dict
                    ret_dict.clear()
                    ret_header.clear()
                    ret_data = ''
                    hasHeader = False
                    
                # parse next header    
                try:
                    ret_header = parse_header(line)
                    ret_dict.update(ret_header)
                    hasHeader = True
                except:
                    ret_header.clear()
                    ret_dict.clear()
                    ret_data = ''
                    hasHeader = False
                    
            else:
                # hex data line
                if hasHeader:
                    data = parse_data(line)
                    ret_data = ret_data + data
                else:
                    continue
    
def is_clean_packet(packet):
    """
    Returns whether or not the parsed packet is valid
    or not. Checks that both the src and dest
    ports are integers. Checks that src and dest IPs
    are valid address formats. Checks that packet data
    is hex. Returns True if all tests pass, False otherwise.
    """
    if not packet['src_port'].isdigit(): return False
    if not packet['dest_port'].isdigit(): return False
    
    if packet['src_ip'].isalpha(): return False
    if packet['dest_ip'].isalpha(): return False
    #try:
    #    ipaddress.ip_address(packet['src_ip'])
    #    ipaddress.ip_address(packet['dest_ip'])
    #except:
    #    return False
     
    if 'data' in packet:
        try:
            int(packet['data'], 16)
        except:
            return False
    
    return True

def order_keys(hexSessionDict):
    orderedKeys = []
    
    for key in sorted(hexSessionDict.keys(), key=lambda key: hexSessionDict[key][1]):
        orderedKeys.append(key) 
        
    return orderedKeys

def read_pcap(path):
    hex_sessions = {}
    proc = subprocess.Popen('tcpdump -nn -tttt -xx -r '+path,
                            shell=True,
                            stdout=subprocess.PIPE)
    insert_num = 0  # keeps track of insertion order into dict
    for packet in process_packet(proc.stdout):
        if not is_clean_packet(packet):
            continue
        if 'data' in packet:
            key = (packet['src_ip']+":"+packet['src_port'], packet['dest_ip']+":"+packet['dest_port'])
            rev_key = (key[1], key[0])
            if key in hex_sessions:
                hex_sessions[key][0].append(packet['data'])
            elif rev_key in hex_sessions:
                hex_sessions[rev_key][0].append(packet['data'])
            else:
                hex_sessions[key] = ([packet['data']], insert_num)
                insert_num += 1
        
    return hex_sessions

In [3]:
def pickleFile(thing2save, file2save2 = None, filePath='/work/notebooks/drawModels/', fileName = 'myModels'):
    
    if file2save2 == None:
        f=file(filePath+fileName+'.pickle', 'wb')
    else:
        f=file(filePath+file2save2, 'wb')
        
    cPickle.dump(thing2save, f, protocol=cPickle.HIGHEST_PROTOCOL)

    f.close()
    
def loadFile(filePath):
    file2open = file(filePath, 'rb')
    loadedFile = cPickle.load(file2open)
    file2open.close()
    
    return loadedFile

In [4]:
def removeBadSessionizer(hexSessionDict, saveFile=False, dataPath=None, fileName=None):
    for ses in hexSessionDict.keys():
        paclens = []
        for pac in hexSessionDict[ses][0]:
            paclens.append(len(pac))
        if np.min(paclens)<80:
            del hexSessionDict[ses]

    if saveFile:
        print 'pickling sessions dictionary... mmm'
        pickleFile(hexSessionDict, filePath=dataPath, fileName=fileName)
        
        #with open(dataPath+'/'+fileName+'.pickle', 'wb') as handle:
        #    cPickle.dump(hexSessions, handle)
            
    return hexSessionDict

In [5]:
dataPath = '/data/fs4/datasets/pcaps/gregPcaps/'
dirList = os.listdir('/data/fs4/datasets/pcaps/gregPcaps/')
dirList

['NESTthermostat-nf-10days-96bytes.pcap',
 'b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan294.pcap',
 'b-dc-24hrs-96bytes-E-FT-SRV-FW1A-2016-08-09_18-16-vlan210.pcap',
 'a-printers-24hrs-96bytes-E-VA-SRV-FW1A-2016-08-09_14-17-vlan34.pcap',
 'a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap',
 'b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan210.pcap',
 'SonySmartTV-nf-10days-96bytes.pcap',
 'TiVoSeries4-nf-10days-96bytes.pcap']

In [6]:
!pwd

/data/fs4/home/bradh


In [8]:
complicated = {}

for capture in dirList:
    dictName = capture.split('.')[0]
    
    start = time.time()
    
    hexSessions = read_pcap(dataPath+capture)
    hexSessions = removeBadSessionizer(hexSessions)
    complicated[dictName] = hexSessions
    
    end = time.time()
    
    print dictName + '  is done'
    print 'time to run (secs): ', (end - start)
    
    
pickleFile(complicated, filePath='/data/fs4/home/bradh/', fileName='complicated')

NESTthermostat-nf-10days-96bytes  is done
time to run (secs):  4.83429908752
b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan294  is done
time to run (secs):  53.9436299801
b-dc-24hrs-96bytes-E-FT-SRV-FW1A-2016-08-09_18-16-vlan210  is done
time to run (secs):  16.7317349911
a-printers-24hrs-96bytes-E-VA-SRV-FW1A-2016-08-09_14-17-vlan34  is done
time to run (secs):  108.721567869
a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40  is done
time to run (secs):  117.949625015
b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan210  is done
time to run (secs):  79.8081531525
SonySmartTV-nf-10days-96bytes  is done
time to run (secs):  49.5780699253
TiVoSeries4-nf-10days-96bytes  is done
time to run (secs):  45.484349966


with open('complicated.pickle', 'rb') as unhandle:
     testDict= cPickle.load(unhandle)

In [12]:
sess = 0
for di in complicated.keys():
    sess += len(complicated[di].keys())
    print di, "  ", len(complicated[di].keys())

print sess

b-dc-24hrs-96bytes-E-FT-SRV-FW1A-2016-08-09_18-16-vlan210    116427
NESTthermostat-nf-10days-96bytes    4933
a-printers-24hrs-96bytes-E-VA-SRV-FW1A-2016-08-09_14-17-vlan34    22029
SonySmartTV-nf-10days-96bytes    147236
a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40    9250
TiVoSeries4-nf-10days-96bytes    2121
b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan210    365186
b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan294    86524
753706


In [ ]:
testDict['a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap']

In [ ]:
tester = read_pcap('/data/fs4/datasets/pcaps/gregPcaps/NESTthermostat-nf-10days-96bytes.pcap')

In [ ]:
len(tester)